In [ ]:
import json
import pandas as pd

import argparse
import io
import os
from datetime import date
# Imports the Google News api
from newsapi import NewsApiClient

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


# Functions

In [ ]:
def classify(text):
    language_client = language.LanguageServiceClient.from_service_account_json('htn2019-f059d4b46f7d.json')
    document = language.types.Document(content=text, type=language.enums.Document.Type.PLAIN_TEXT)
    response = language_client.classify_text(document)
    categories = response.categories

    result = {}

    for category in categories:
        categories = category.name.split("/")
        categories.pop(0)
        for i in range(len(categories)):
            result[categories[i]] = category.confidence

    return result

## Input Parameter

In [ ]:
print("Enter the original currency: ")
# original = input()
print("Enter the new currency: ")
# return = input()

In [ ]:
original = 'CAD'

## Article Retrieval 

In [ ]:
newsapi = NewsApiClient(api_key='a93fd9a824b848bca3436e761c5a0d4d')

In [ ]:
today = date.today().strftime("%Y/%m/%d").replace("/","-")

# grabbing articles
all_articles = newsapi.get_everything(q=original, from_param='2019-08-15', to=today, language='en', sort_by='relevancy')
articles = all_articles['articles']

In [ ]:
formatted_articles = []

# creating new article object
for article in articles:
    info = str(article['title']) + " " + str(article['description']) + " " + str(article['content'])
    classifications = classify(info)
    print(classifications)
    print()
#     print(classifications)
    if classifications:
        individual_article = {}
        # just the date
        # print(article['publishedAt'][:10])
        individual_article['date'] = article['publishedAt']

        info = info.replace("'","") 
        individual_article['text'] = info

        formatted_articles.append(individual_article)


## Sentiment Analysis (Tone Analyzer)

In [ ]:
client = language.LanguageServiceClient.from_service_account_json('htn2019-f059d4b46f7d.json')

In [ ]:
tone_analyzed_articles = []

# The text to analyze
for i in range(len(formatted_articles)):
    analyzed_article = {}
    article = formatted_articles[i]
    text = article['text']
    
    document = types.Document(content = text, type = enums.Document.Type.PLAIN_TEXT)
    
    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    
    analyzed_article['date'] = article['date']
    analyzed_article['score'] = sentiment.score
    analyzed_article['magnitude'] = sentiment.magnitude
    
    tone_analyzed_articles.append(analyzed_article)

print(json.dumps(tone_analyzed_articles, indent=4, sort_keys=True))

#     print('Text: {}'.format(text))
#     print('Sentiment: {}, {}'.format(, sentiment.magnitude))